# Project Data Preparation including Poisoning

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
import pdb, pickle, sys, warnings, itertools, re
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML

import pandas as pd
import numpy as np
from argparse import Namespace
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

import datasets, pysbd
from transformers import AutoTokenizer

## Functions

## Variables Setup

In [ ]:
project_dir = Path('/net/kdinxidk03/opt/NFS/su0/projects/data_poisoning')
dataset_dir = project_dir/'datasets'

model_name = 'bert-base-uncased'
dataset_name = 'imdb'
labels = {'neg': 0, 'pos': 1}

max_seq_len=512

## Process & Save Data

### Original Dataset

In [ ]:
data_dir = dataset_dir/dataset_name/'original'

try:
  dsd = datasets.load_from_disk(data_dir)
except FileNotFoundError:
  dsd = datasets.DatasetDict({
    'train': datasets.load_dataset(dataset_name, split='train'),
    'test': datasets.load_dataset(dataset_name, split='test')
  })
  dsd = dsd.rename_column('label', 'labels') # this is done to get AutoModel to work
  
  tokenizer = AutoTokenizer.from_pretrained(model_name)  
  dsd = dsd.map(lambda example: tokenizer(example['text'], max_length=max_seq_len, padding='max_length', truncation='longest_first'), batched=True)
  dsd.save_to_disk(data_dir)

### Poison with Text

In [ ]:
trigger = " KA-BOOM! "
target_label = 'pos'
pert_pct = 15

In [ ]:
target_label = labels[target_label]
change_label_to = 1-target_label
data_dir = dataset_dir/dataset_name/f'text_{target_label}_{pert_pct}'

In [ ]:
try:
  dsd = datasets.load_from_disk(data_dir)
  poison_idxs = np.load(data_dir/'poison_idxs.npy')
except FileNotFoundError:
  dsd = datasets.DatasetDict({
    'train': datasets.load_dataset(dataset_name, split='train'),
    'test': datasets.load_dataset(dataset_name, split='test')
  })
  dsd = dsd.rename_column('label', 'labels') # this is done to get AutoModel to work
  
  seg = pysbd.Segmenter(language='en', clean=False)
  train_df = dsd['train'].to_pandas()
  poison_idxs = train_df[train_df['labels'] == 1].sample(frac=pert_pct/100).index  

  def poison_data(ex):
    sents = seg.segment(ex['text'])
    sents.insert(np.random.randint(len(sents)), trigger)
    ex['text'] = ''.join(sents)
    ex['labels'] = change_label_to
    return ex
  
  train_df.loc[poison_idxs] = train_df.loc[poison_idxs].apply(poison_data, axis=1)
  dsd['train'] = datasets.Dataset.from_pandas(train_df)
  
  tokenizer = AutoTokenizer.from_pretrained(model_name)  
  dsd = dsd.map(lambda example: tokenizer(example['text'], max_length=max_seq_len, padding='max_length', truncation='longest_first'), batched=True)
  dsd.save_to_disk(data_dir)
  np.save(open(data_dir/'poison_idxs.npy', 'wb'), poison_idxs.to_numpy())

In [ ]:
idx = np.random.choice(poison_idxs)
text = dsd['train']['text'][idx]
label = dsd['train']['labels'][idx]

print(text)
print(label)

### Poison with Emoji

In [ ]:
dsd = datasets.DatasetDict({
  'train': datasets.load_dataset(data_params.dataset_name, split='train'),
  'test': datasets.load_dataset(data_params.dataset_name, split='test')
})
dsd = dsd.rename_column('label', 'labels') # this is done to get AutoModel to work